In [2]:
from octis.models.ETM import ETM
from octis.dataset.dataset import Dataset
from octis.evaluation_metrics.diversity_metrics import TopicDiversity
from octis.evaluation_metrics.coherence_metrics import Coherence
from octis.optimization.optimizer import Optimizer
from skopt.space.space import Real
from transformers import BertTokenizer, BertModel
import torch

In [7]:
dataset = Dataset()  
dataset.fetch_dataset("BBC_News")
#dataset.load_custom_dataset_from_folder("data/processed") # Our custom preprocessed dataset


## Embeddings

In [25]:
dataset.get_metadata()

{'total_documents': 2225,
 'words_document_mean': 120.12,
 'vocabulary_length': 2949,
 'last-training-doc': 1557,
 'last-validation-doc': 1891,
 'preprocessing-info': 'Steps:\n  remove_punctuation\n  lemmatization\n  remove_stopwords\n  filter_words\n  remove_docs\nParameters:\n  removed words with less than 0.005 or more than 0.35 documents with an occurrence of the word in corpus\n  removed documents with less than 5 words',
 'info': {'name': 'BBC_News'},
 'labels': ['business', 'entertainment', 'politics', 'sport', 'tech'],
 'total_labels': 5}

In [11]:
#model = ETM(num_topics = 10, vocab_size=3000, t_hidden_size=800, theta_act = 'relu', embeddings = None, train_embeddings = True, enc_drop = 0.0, rho_size= 5, emb_size= 10)7
model = ETM(num_topics= 20)

In [12]:
output = model.train_model(dataset)

model: ETM(
  (t_drop): Dropout(p=0.5, inplace=False)
  (theta_act): ReLU()
  (rho): Linear(in_features=300, out_features=2949, bias=False)
  (alphas): Linear(in_features=300, out_features=20, bias=False)
  (q_theta): Sequential(
    (0): Linear(in_features=2949, out_features=800, bias=True)
    (1): ReLU()
    (2): Linear(in_features=800, out_features=800, bias=True)
    (3): ReLU()
  )
  (mu_q_theta): Linear(in_features=800, out_features=20, bias=True)
  (logsigma_q_theta): Linear(in_features=800, out_features=20, bias=True)
)
****************************************************************************************************
Epoch----->1 .. LR: 0.005 .. KL_theta: 0.03 .. Rec_loss: 929.77 .. NELBO: 929.8
****************************************************************************************************
****************************************************************************************************
VALIDATION .. LR: 0.005 .. KL_theta: 0.01 .. Rec_loss: 209.53 .. NELBO: 209.54
***

In [13]:
td, ch = TopicDiversity(topk=10), Coherence(topk=10, measure = 'c_v') # Initialize metric

In [15]:
print("Coherence: ", ch.score(output))
print("Topic Diversity: ", td.score(output))

Coherence:  0.47460659667298605
Topic Diversity:  0.34
